[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hemanth1403/IIITH-Student-training-program-AIML/blob/main/AIML_Module_1_Lab_2_Machine_learning_terms_and_metrics.ipynb)

##AIML Module 1 - Lab 2
# Machine learning terms and metrics


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)
print(rng)

Generator(PCG64)


In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
dataset =  datasets.fetch_california_housing()
# print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
# print(dataset.keys())  # if you want to know what else is there in this dataset
dataset.target = dataset.target.astype(int) # so that we can classify
print(dataset.data.shape)
print(dataset.target.shape)
# print(dataset)

(20640, 8)
(20640,)


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
# print('Percent of val data = ', len(vallabel)*100/len(alltrainlabel),'%')

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

In [ ]:
# 1. A.
# If we Decrease the validation size the validation accuracy Increases
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 95/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

# If we Increase the validation size the validation accuracy Decreases
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 55/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.3471882640586797
Validation accuracy of nearest neighbour is  0.33369156367544334


In [ ]:
# 2. A.
# 50 - 30 - 20 split i.e 50 Training, 30 Validation, 20 Testing
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 63/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)
print('Test accuracy is ', testAccuracy)
print("-------------")
# 80 - 10 - 10 split i.e 80 Training, 10 Validation, 10 Testing
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 10/100)
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 89/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)
print('Test accuracy is ', testAccuracy)
print("-------------")
# 60 - 20 - 20 split i.e 60 Training, 20 Validation, 20 Testing
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)
print('Test accuracy is ', testAccuracy)



Validation accuracy of nearest neighbour is  0.3438428524265434
Test accuracy is  0.3466666666666667
-------------
Validation accuracy of nearest neighbour is  0.35410764872521244
Test accuracy is  0.3437959784207945
-------------
Validation accuracy of nearest neighbour is  0.3502921129503408
Test accuracy is  0.35195396787341166


3. A. Good percentage to reserve for the validation set depends on the dataset, For nomal dataset it can be around 70(Train) - 15(Val) - 15(Test) so that thest two factors are balanced and In case of large datasets it can be 80(Train) - 10(Val) - 10(Test) so that these two factors are balacned.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33983283755624494
test accuracy is  0.35195396787341166


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1. A. Yes, especially when utilizing methods like k-fold cross-validation, averaging the validation accuracy across numerous splits can lead to more dependable and consistent results. This method mitigates the effects of a single random split of the data, which may be skewed or not representative of the dataset as a whole and reduces the variance in our model evaluation.

2. A. Compared to using just one train-test split, k-fold cross-validation delivers a more accurate estimate of the model's expected performance on unseen data by averaging validation accuracy across numerous splits. It's crucial to remember that the validation accuracy is still only a rough estimate of the test accuracy; it differs significantly from the test accuracy you would actually achieve by applying your model to completely new data.

In [ ]:
# 3. A. Up to a certain point, increasing the number of iterations can generally result in a better assessment of a model's performance.
# There are diminishing returns, though, as extremely high iteration counts can result in longer calculation times without much gain.

# For 2 iterations
print('Average validation accuracy is for 2 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 2, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 5 iterations
print('Average validation accuracy is for 5 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 5, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 8 iterations
print('Average validation accuracy is for 8 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 8, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 10 iterations
print('Average validation accuracy is for 10 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 12 iterations
print('Average validation accuracy is for 12 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 12, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 15 iterations
print('Average validation accuracy is for 15 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 15, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 18 iterations
print('Average validation accuracy is for 18 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 18, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 20 iterations
print('Average validation accuracy is for 20 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 20, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 50 iterations
print('Average validation accuracy is for 50 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 50, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 80 iterations
print('Average validation accuracy is for 80 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 80, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))
print("-------")
# For 100 iterations
print('Average validation accuracy is for 100 iterations : ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 100, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred))


Average validation accuracy is for 2 iterations :  0.3405614461870053
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 5 iterations :  0.34037876663413014
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 8 iterations :  0.34361144387031795
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 10 iterations :  0.34228543340334106
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 12 iterations :  0.34447072141355517
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 15 iterations :  0.3417489640181012
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 18 iterations :  0.3434763764767076
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 20 iterations :  0.33896892115755173
test accuracy is  0.35195396787341166
-------
Average validation accuracy is for 50 iterations :  0.34153143

4. A. To some extent, increasing the number of training iterations can help a model learn more from a limited training dataset. However, relying solely on increasing iterations when dealing with a very small training dataset can lead to Overfitting risks, Diminishing returns and bias-variance trade-off.